In [8]:
import os 

import torch
import torch.nn as nn 
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from helpers import load_experiment

In [9]:
dm, model = load_experiment("/home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-03/04-28-27")

Fetching configuration...
Loading datamodule...


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 2700/2700 [00:00<00:00, 8971.89it/s]


Len of vocab:  54
Max len of caption:  16
Index for <pad>: [0]
Loading model


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(


In [10]:
train_loader = dm.train_dataloader()
train_set = train_loader.dataset

In [11]:
# [red, green, blue, square, triangle, circle]

attribute_mapping = {
    0: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0
        ],
    1: [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
        ],
    2: [0, 0, 1, 1, 0, 0,  0, 0, 1, 0, 0, 0, 0, 0, 0
        ],
    3: [1, 0, 0, 0, 1, 0,   0, 0, 0, 1, 0, 0, 0, 0, 0
        ],
    4: [0, 1, 0, 0, 1, 0,   0, 0, 0, 0, 1, 0, 0, 0, 0
        ],
    5: [0, 0, 1, 0, 1, 0,   0, 0, 0, 0, 0, 1, 0, 0, 0
        ],
    6: [1, 0, 0, 0, 0, 1,   0, 0, 0, 0, 0, 0, 1, 0, 0
        ],
    7: [0, 1, 0, 0, 0, 1,   0, 0, 0, 0, 0, 0, 0, 1, 0
        ],
    8: [0, 0, 1, 0, 0, 1,   0, 0, 0, 0, 0, 0, 0, 0, 1
        ],
}

In [12]:
sample = train_set[50]
image = sample["image"]

image = image.unsqueeze(0).cuda()


# model.main.inference(image)[1].cpu().detach().numpy()[0]
model(image)["concept_probs"].cpu().detach().numpy()[0]

array([9.6600628e-01, 9.0756133e-04, 9.7032261e-01, 1.7928847e-03,
       4.8029178e-01, 1.3788255e-04, 3.4612307e-04, 2.1299124e-03,
       9.7708470e-01, 4.2304207e-04], dtype=float32)

In [13]:
scores = list()

for feature_id in range(10):

    attribute_values = [[0, 0] for _ in range(len(attribute_mapping[0]))]

    for i in tqdm(range(len(train_set))):
        sample = train_set[i]
        
        idx = sample["target"].item()
        attributes = np.array(attribute_mapping[idx])

        image = sample["image"]
        image = image.unsqueeze(0).cuda()

        # feature = model.main.inference(image)[1].cpu().detach().numpy()[0][feature_id]
        feature = model(image)["concept_probs"].cpu().detach().numpy()[0][feature_id]

        for attr_id, attribute in enumerate(attributes):
            value_on = attribute * feature + (1 - attribute) * (1 - feature)
            attribute_values[attr_id][0] += value_on

            value_off = attribute * (1 - feature) + (1 - attribute) * feature
            attribute_values[attr_id][1] += value_off
    
    attribute_values_max = [max(p) for p in attribute_values]
    attribute_values_max = max(attribute_values_max)
    score = attribute_values_max / len(train_set)

    scores.append(score)

metric = np.array(scores).mean()

print(metric)

100%|██████████| 1845/1845 [00:34<00:00, 53.50it/s]

0.8002787673469156


Baseline: 0.7481238945043126 / 0.8002787673469156

Our framework: 0.733698356318566 / 0.733698356318566


In [14]:
scores # baseline

[0.8774386084511677,
 0.8683422300754077,
 0.708308124278052,
 0.7367455025824375,
 0.9301602215365251,
 0.819872051679877,
 0.7686549335254693,
 0.6847247716806912,
 0.8782771141074441,
 0.7302641155520837]

In [7]:
scores # framework

[0.7589874283948201,
 0.6796973172001046,
 0.7047002247698746,
 0.7227437217141878,
 0.7557682150717431,
 0.717886862786093,
 0.6422506623823501,
 0.7989227409521245,
 0.8816705980478233,
 0.674355791866537]